In [32]:
import requests
import datetime
import json

LCD_ENDPOINT_MAIN = "https://fcd.terra.dev/v1/txs/"
CONTRACT = "terra1kc87mu460fwkqte29rquh4hc20m54fxwtsx7gp"
def fetch_txs(LCD_ENDPOIN,CONTRACT):
    txs = []
    params = {
        "account": CONTRACT,
        "limit": 100
    }
    i = 0
    until_day = datetime.date(2021, 8, 15)
    day = datetime.date.today()
    while day >= until_day:
    # while i<1:
        resp = requests.get(LCD_ENDPOIN, params=params)
        txs = txs + resp.json()['txs']
        n = resp.json()['next']
        params['offset'] = n
        i += 1
        day = datetime.datetime.strptime(
            resp.json()['txs'][0]["timestamp"], "%Y-%m-%dT%H:%M:%SZ").date()
        print(resp.json()['txs'][0]["timestamp"])

    valuable_txs = []
    for tx in txs:
        tx.pop('logs', None)
        tx.pop('raw_log', None)
        # if get_uluna(tx,CONTRACT) >0:
        #     valuable_txs.append(tx)
    return txs

txs = fetch_txs(LCD_ENDPOINT_MAIN,CONTRACT)
fd = open("txs-dump-bluna.json", "w")
fd.write(json.dumps(txs))
fd.close()

2021-08-25T13:36:26Z


In [36]:
import copy
CONTRACT = "terra1kc87mu460fwkqte29rquh4hc20m54fxwtsx7gp"
fd = open("txs-dump-bluna-full-messages.json")
txs = json.loads(fd.read())


def parse_day(date):
    return datetime.datetime.strptime(date, "%Y-%m-%dT%H:%M:%SZ").date()

# message
# sender
# amount
# date


def get_send_message_from_logs(tx):
    send_msg = {}
    msgs = []
    type_detect_field = "action"
    for log in tx['logs']:
        for e in log['events']:
            if e['type'] != "from_contract":
                continue
            match_contract = False
            match_message_type = False
            for a in e['attributes']:
                if a['key'] == "contract_address":
                    if match_contract and match_message_type:
                        msgs.append(copy.deepcopy(send_msg))
                    send_msg = {}
                    match_contract = False
                    match_message_type = False
                    if a['value'] == CONTRACT:
                        match_contract = True
                    continue
                if match_contract and a['key'] == type_detect_field and a['value'] == 'send':
                    match_message_type = True
                    continue
                if match_contract and match_message_type:
                    send_msg[a['key']] = a['value']
            if match_contract and match_message_type:
                msgs.append(copy.deepcopy(send_msg))
    return msgs


for tx in txs:
    msgs = get_send_message_from_logs(tx)
    if len(msgs) > 0:
        print(msgs)


[{'from': 'terra13n92s0u0syt5u82pxgryetdurde237shhk8u0y', 'to': 'terra1jxazgm67et0ce260kvrpfv50acuushpjsz2y0p', 'amount': '500000000'}]
[{'from': 'terra13n92s0u0syt5u82pxgryetdurde237shhk8u0y', 'to': 'terra1jxazgm67et0ce260kvrpfv50acuushpjsz2y0p', 'amount': '500000000'}]
[{'from': 'terra14w7ellwa9pwld8efd4s9gh835gvr0pn0lnn96g', 'to': 'terra1mtwph2juhj0rvjz7dy92gvl6xvukaxu8rfv8ts', 'amount': '999992'}]
[{'from': 'terra1han7mqkrv75q2rffmd3vy06d6culn4s7j95xn0', 'to': 'terra1ptjp2vfjrwh0j0faj9r6katm640kgjxnwwq9kn', 'amount': '49999640'}]
[{'from': 'terra1xn35q6ej2dxru70evnmq2zhfdx2m6r0hqgc0fc', 'to': 'terra1ptjp2vfjrwh0j0faj9r6katm640kgjxnwwq9kn', 'amount': '185241710'}]
[{'from': 'terra1jx4x20k9648angmf30lssu677yaxq634w7a0hw', 'to': 'terra1ptjp2vfjrwh0j0faj9r6katm640kgjxnwwq9kn', 'amount': '77281491'}]
[{'from': 'terra13sq3zm8gt329cvtq48s89krl0z32wwq3gwz7cm', 'to': 'terra1mtwph2juhj0rvjz7dy92gvl6xvukaxu8rfv8ts', 'amount': '100000000'}]
[{'from': 'terra1kk6uw6xgr70rnp0s6c5m9tgjgqdakxuezpd6

In [29]:
from collections import defaultdict
import base64
import json
import datetime
import plotly
import plotly.graph_objs as go
import plotly.graph_objects as px
from plotly.subplots import make_subplots
CONTRACT = "terra1kc87mu460fwkqte29rquh4hc20m54fxwtsx7gp"
fd = open("txs-dump-bluna.json")
txs = json.loads(fd.read())


def add_date(m, date):
    m['date'] = date
    return m


def find_related_messages(txs, contract):
    msgs = []
    for tx in txs:
        msgs = msgs + [add_date(msg, tx['timestamp']) for msg in tx['tx']
                       ['value']['msg'] if msg['value']['contract'] == contract]
    return msgs


def get_message_type(base64_msg):
    decoded_m = json.loads(base64.b64decode(base64_msg))
    return list(decoded_m.keys())[0]


def get_amount_from_send(base64_msg):
    decoded_m = json.loads(base64.b64decode(base64_msg))
    return decoded_m['send']['amount']


class SendMessage(object):
    def __init__(self, amount, date, sender, sub_message_type):
        self.amount = int(amount)
        self.date = datetime.datetime.strptime(date, "%Y-%m-%dT%H:%M:%SZ").date()
        self.sender = sender
        # one of unbond, swap, deposit_collateral
        # '{"swap":{"belief_price":"1.049714260737302397","max_spread":"0.1"}}'
        # '{"deposit_collateral":{}}'
        # '{"unbond":{}}'
        self.sub_message_type = sub_message_type


send_messages = defaultdict(list)

messages = find_related_messages(txs, CONTRACT)
message_type = defaultdict(int)
send_submessage_type = defaultdict(int)
for m in messages:
    mt = get_message_type(m['value']['execute_msg'])
    message_type[mt] += 1
    if mt == "send":
        smt = get_message_type(json.loads(base64.b64decode(
            m['value']['execute_msg']))['send']['msg'])
        sm = SendMessage(
            get_amount_from_send(m['value']['execute_msg']),
            m['date'],
            m['value']['sender'],
            smt,
        )
        # print(sm.date)
        send_messages[sm.date].append(sm)
        send_submessage_type[smt] += 1


def process_messages(message_type):


    # subtype = filter(lambda s: s.sub_message_type=='message_type',send_messages)
    subtype_days = []
    subtype_days_amount = []
    subtype_days_count = []
    for day,message_list in send_messages.items():
        subtype_days.append(day)
        subtype_days_amount.append(
            sum([m.amount for m in message_list if m.sub_message_type==message_type])
        )
        subtype_days_count.append(
            len([m for m in message_list if m.sub_message_type==message_type])
        )
    return subtype_days,subtype_days_amount,subtype_days_count

subtype_swap_days,subtype_swap_days_amount,subtype_swap_days_count = process_messages('swap')
subtype_unbond_days,subtype_unbond_days_amount,subtype_unbond_days_count = process_messages('unbond')
subtype_deposit_collateral_days,subtype_deposit_collateral_days_amount,subtype_deposit_collateral_days_count = process_messages('deposit_collateral')


In [31]:
for message_type in ['swap','unbond','deposit_collateral']:
    subtype_days,subtype_days_amount,subtype_days_count = process_messages(message_type)
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=subtype_days, y=subtype_days_amount, name=f"Суммы по дням {message_type}"),
                secondary_y=False)
    fig.add_trace(go.Scatter(x=subtype_days, y=subtype_days_count,
                            name=f"Кол-во сообщений {message_type}"), secondary_y=True)
    fig.show()


In [31]:
import requests
import datetime
import json
import plotly
import plotly.graph_objs as go
import plotly.graph_objects as px
from collections import defaultdict
from plotly.subplots import make_subplots
import base64
from statistics import stdev,mean

LCD_ENDPOINT = "https://tequila-fcd.terra.dev/v1/txs/"
CONTRACT = "terra1qzzcc8r04ylr20lwzhlafwynqtmjdnrzlc35jh"

LCD_ENDPOINT = "https://fcd.terra.dev/v1/txs/"
CONTRACT = "terra1mtwph2juhj0rvjz7dy92gvl6xvukaxu8rfv8ts"


def fetch_txs():
    txs = []
    params = {
        "account": CONTRACT,
        "limit": 100
    }
    i = 0
    until_day = datetime.date(2021, 5, 18)
    day = datetime.date.today()
    while day >= until_day:
        resp = requests.get(LCD_ENDPOINT, params=params)
        txs = txs + resp.json()['txs']
        n = resp.json()['next']
        params['offset'] = n
        i += 1
        day = datetime.datetime.strptime(
            resp.json()['txs'][0]["timestamp"], "%Y-%m-%dT%H:%M:%SZ").date()
        print(resp.json()['txs'][0]["timestamp"])

    valuable_txs = []
    for tx in txs:
        tx.pop('logs', None)
        tx.pop('raw_log', None)
        # if get_uluna(tx,CONTRACT) >0:
        #     valuable_txs.append(tx)
    return txs


def get_uluna(tx, contract):
    coins = [(msg['value']['coins'],(base64.b64decode(msg['value']['execute_msg']).decode("utf-8"))) for msg in tx['tx']['value']['msg']
             if 'coins' in msg['value'] and msg['value']['contract'] == contract]
    if len(coins) > 0 and len(coins[0][0]) > 0:
        if coins[0][0][0]['denom'] == 'uluna':
            return (int(coins[0][0][0]['amount']),coins[0][1])
    return (0,"")



def funds_per_day(tx_list):
    funds = defaultdict(lambda: defaultdict(int))
    send_number = defaultdict(int)
    for tx in tx_list:
        day = datetime.datetime.strptime(
            tx["timestamp"], "%Y-%m-%dT%H:%M:%SZ").date()
        uluna,msg = get_uluna(tx, CONTRACT)
        funds[day][msg] = funds[day][msg] + uluna
        send_number[day] += 1
    return funds, send_number


if False:
    txs = fetch_txs()
    fd = open("txs-dump.json", "w")
    fd.write(json.dumps(txs))
    fd.close()
else:
    fd = open("txs-dump.json")
    txs = json.loads(fd.read())


days = []
sums = []
delegation_list = []
funds, delegations = funds_per_day(txs)
funds_by_message = defaultdict(lambda: defaultdict(int))
for k, v in funds.items():
    days.append(k)
    sums.append(sum(list(v.values())))
    delegation_list.append(delegations[k])
    for msg in v.keys():
        funds_by_message[msg][k] = v[msg]

print(stdev(delegation_list))
print(mean(delegation_list))
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=days, y=sums, name="Суммы по дням"),
              secondary_y=False)
fig.add_trace(go.Scatter(x=days, y=delegation_list,
                         name="Кол-во делегаций"), secondary_y=True)
fig.show()

bars = []
for msg,data in funds_by_message.items():
    days = list(data.keys())
    coins = list(data.values())
    bars.append(
        go.Bar(
            name=msg
        ,x=days,y=coins)
    )
plot = px.Figure(data=bars)

plot.update_layout(barmode='stack')

plot.show()


326.6534592747188
788.5416666666666
